In [1]:
import os
from typing import TypedDict

import markdown
import pandas as pd
import pymysql
import requests
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langgraph.graph import END, START, StateGraph

llm = ChatOpenAI(model="gpt-4o-mini")
#os.environ["LANGCHAIN_TRACING_V2"] = "true"
#os.environ["LANGCHAIN_PROJECT"] = "SEO_cipcode"

major_students_url = "https://www.forwardpathway.com/d3v7/dataphp/CIPCODE/CIPCODE_yearly_20230923.php?CIPCODE="
major_colleges_url = """https://www.forwardpathway.com/d3v7/dataphp/CIPCODE/CIPCODE_schools_20230920.php"""

In [2]:
def get_cipcode_list():
    connection = pymysql.connect(
        db=os.environ["db_name"],
        user=os.environ["db_user"],
        passwd=os.environ["db_pass"],
        host=os.environ["db_host"],
        port=3306,
        cursorclass=pymysql.cursors.DictCursor,
    )
    cursor = connection.cursor()
    query = "SELECT CIPCODE,major,translation FROM fp_IPEDS.major_translation"
    cursor.execute(query)
    rows = cursor.fetchall()
    majors = pd.DataFrame(
        columns=[
            "CIPCODE",
            "major",
            "translation",
        ]
    )
    for row in rows:
        majors = pd.concat([majors, pd.DataFrame([row])])
    majors = majors.reset_index(drop=True)
    cursor.close()
    connection.close()
    return majors

In [3]:
class cipcodeState(TypedDict):
    index: int
    cipcode: str
    ename: str
    cname: str
    des: str
    des_en: str
    des2: str
    des2_en: str
    des3: str
    des3_en: str

In [4]:
def get_cipcode_name(state):
    index = state["index"]
    cipcode = majors.iloc[index]["CIPCODE"]
    ename = majors.iloc[index]["major"]
    cname = majors.iloc[index]["translation"]
    return {"cipcode": cipcode, "ename": ename, "cname": cname}

In [5]:
def get_major_des(state):
    major_ename = state["ename"]
    major_cname = state["cname"]
    des_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """你是一位了解美国大学各个专业的专家，下面用户给出的是一个专业的中文名和英文名，请就你对该专业的了解做一段简单的介绍，字数在300字左右。""",
            ),
            ("human", "专业中文名：{major_cname}，专业英文名：{major_ename}。"),
        ]
    )
    des_chain = des_prompt | llm | StrOutputParser()
    response = des_chain.invoke(
        {"major_cname": major_cname, "major_ename": major_ename}
    )
    formatted_response = markdown.markdown(response).replace("</p>\n<p>", "</p><p>")
    return {"des": formatted_response}

In [6]:
def get_major_des_en(state):
    major_ename = state["ename"]
    major_cname = state["cname"]
    des_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """你是一位了解美国大学各个专业的专家，下面用户给出的是一个专业的中文名和英文名，请就你对该专业的了解做一段简单的介绍，字数在300字左右。
                    最终数据结果请使用英文。The final output should be in purely English.""",
            ),
            ("human", "专业中文名：{major_cname}，专业英文名：{major_ename}。"),
        ]
    )
    des_chain = des_prompt | llm | StrOutputParser()
    response = des_chain.invoke(
        {"major_cname": major_cname, "major_ename": major_ename}
    )
    formatted_response = markdown.markdown(response).replace("</p>\n<p>", "</p><p>")
    return {"des_en": formatted_response}

In [7]:
def major_students_comments(state):
    major_ename = state["ename"]
    major_cname = state["cname"]
    cipcode = state["cipcode"]
    try:
        reqeust_response = requests.get(major_students_url + cipcode)
        data = reqeust_response.json()
        student_comp_data = data[-1]["data2"]
        major_year = str(data[-1]["year"])
        del data[-1]["data2"]
        students_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """你是一位了解美国大学各个专业的专家，下面用户给出的是一个专业的中文名和英文名，以及该专业在全美各大高校毕业生不同年份的两份数据，请对这两份数据做一个评论，具体要求如下：
                    1. 第一份数据是不同年份不同学位的毕业生人数，其中"year"为年份，"xs"为该年份总毕业生人数，"lxs"为该年份总毕业生中留学生毕业人数，"xs0"为副学士毕业生人数，"lxs0"为副学士留学生毕业生人数，
                        "xs1"为本科毕业生人数，"lxs1"为本科留学生毕业人数，"xs2"为硕士毕业生人数，"lxs2"为硕士留学生毕业人数，"xs3"为博士毕业生人数，"lxs3"为博士留学生毕业生人数。
                    2. 第二份数据是"""
                    + major_year
                    + """年该专业不同学位毕业生人数的人种细分数据，其中"Associate"为副学士学位，"Bachelor"为本科学位，"Master"为硕士学位，"Doctor"为博士学位，
                        人种细分数据中"Whtie"为白人，"Asian"为亚裔，"Hispanic"为西班牙裔，"Pacific and others"为太平洋岛国及其他，"African"为非裔，"International"为国际留学生。
                    3. 请对这两份数据做一个综合的分析评论，评论的字数在500字左右。""",
                ),
                (
                    "human",
                    "专业中文名：{major_cname}，专业英文名：{major_ename}\n\n第一份数据：{data}\n\n第二份数据：{student_comp_data}。",
                ),
            ]
        )
        students_chain = students_prompt | llm | StrOutputParser()
        response = students_chain.invoke(
            {
                "major_cname": major_cname,
                "major_ename": major_ename,
                "data": data,
                "student_comp_data": student_comp_data,
            }
        )
        formatted_response = markdown.markdown(response).replace("</p>\n<p>", "</p><p>")
    except:
        formatted_response = None
    return {"des2": formatted_response}

In [8]:
def major_students_comments_en(state):
    major_ename = state["ename"]
    major_cname = state["cname"]
    cipcode = state["cipcode"]
    try:
        reqeust_response = requests.get(major_students_url + cipcode)
        data = reqeust_response.json()
        student_comp_data = data[-1]["data2"]
        major_year = str(data[-1]["year"])
        del data[-1]["data2"]
        students_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """你是一位了解美国大学各个专业的专家，下面用户给出的是一个专业的中文名和英文名，以及该专业在全美各大高校毕业生不同年份的两份数据，请对这两份数据做一个评论，具体要求如下：
                    1. 第一份数据是不同年份不同学位的毕业生人数，其中"year"为年份，"xs"为该年份总毕业生人数，"lxs"为该年份总毕业生中留学生毕业人数，"xs0"为副学士毕业生人数，"lxs0"为副学士留学生毕业生人数，
                        "xs1"为本科毕业生人数，"lxs1"为本科留学生毕业人数，"xs2"为硕士毕业生人数，"lxs2"为硕士留学生毕业人数，"xs3"为博士毕业生人数，"lxs3"为博士留学生毕业生人数。
                    2. 第二份数据是"""
                    + major_year
                    + """年该专业不同学位毕业生人数的人种细分数据，其中"Associate"为副学士学位，"Bachelor"为本科学位，"Master"为硕士学位，"Doctor"为博士学位，
                        人种细分数据中"Whtie"为白人，"Asian"为亚裔，"Hispanic"为西班牙裔，"Pacific and others"为太平洋岛国及其他，"African"为非裔，"International"为国际留学生。
                    3. 请对这两份数据做一个综合的分析评论，评论的字数在400字左右，最终输出结果必输是纯英文，the final output should be in purely English。""",
                ),
                (
                    "human",
                    "专业中文名：{major_cname}，专业英文名：{major_ename}\n\n第一份数据：{data}\n\n第二份数据：{student_comp_data}。",
                ),
            ]
        )
        students_chain = students_prompt | llm | StrOutputParser()
        response = students_chain.invoke(
            {
                "major_cname": major_cname,
                "major_ename": major_ename,
                "data": data,
                "student_comp_data": student_comp_data,
            }
        )
        formatted_response = markdown.markdown(response).replace("</p>\n<p>", "</p><p>")
    except:
        formatted_response = None
    return {"des2_en": formatted_response}

In [9]:
def major_colleges_comments(state):
    major_ename = state["ename"]
    major_cname = state["cname"]
    cipcode = state["cipcode"]
    try:
        reqeust_response = requests.get(major_colleges_url + "?CIPCODE=" + cipcode)
        data = reqeust_response.json()
        major_year = data["year"]
        rank_year = data["rank_year"]
        data = data["children"]
        students_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """你是一位了解美国大学各个专业的专家，下面用户给出的是一个专业的中文名和英文名，以及该专业在全美各大高校"""
                    + major_year
                    + """年不同学位的毕业生人数，请对这份数据做一个评论，具体要求如下：
                    1. 首先数据分为6大块，分别包含"""
                    + rank_year
                    + """年USNews美国大学排名1-50,51-100,101-150,151-200,201-250以及251-300排名区间的有该专业毕业生的大学。
                    2. 其次，每个排名区间下面包含各大学的该年份毕业生人数，其中"name"为大学中文名，"ename"为大学英文名，"postid"为大学代码，"rank"为"""
                    + rank_year
                    + """年USNews美国大学排名，"count"为该校该专业毕业生总人数，
                        "count0"为该专业该校副学士毕业人数，"count1"为该专业该校本科毕业人数，"count2"为该专业该校硕士毕业人数，"count3"为该专业该校博士毕业人数。
                    3. 请对这份数据做一个综合的分析评论，评论中请挑选一些重点大学评价，比如毕业生人数比较多的大学或者排名比较高的大学，或者有其他特点的大学等等，评论的字数在800字左右。
                    4. 对评论中出现的美国大学请只使用中文名，中文名上需要加链接，链接名称为学校中文名，链接url为"https://www.forwardpathway.com/postid"，其中"postid"为数据中该大学"postid"项。
                    5. 评论输出的格式不要包含标题，只用段落的方式输出内容。""",
                ),
                (
                    "human",
                    "专业中文名：{major_cname}，专业英文名：{major_ename}\n\n具体数据：{data}",
                ),
            ]
        )
        students_chain = students_prompt | llm | StrOutputParser()
        response = students_chain.invoke(
            {
                "major_cname": major_cname,
                "major_ename": major_ename,
                "data": data,
            }
        )
        formatted_response = markdown.markdown(response).replace("</p>\n<p>", "</p><p>")
    except:
        formatted_response = None
    return {"des3": formatted_response}

In [10]:
def major_colleges_comments_en(state):
    major_ename = state["ename"]
    major_cname = state["cname"]
    cipcode = state["cipcode"]
    try:
        reqeust_response = requests.get(
            major_colleges_url + "?v=fpus&CIPCODE=" + cipcode
        )
        data = reqeust_response.json()
        major_year = data["year"]
        rank_year = data["rank_year"]
        data = data["children"]
        students_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    """你是一位了解美国大学各个专业的专家，下面用户给出的是一个专业的中文名和英文名，以及该专业在全美各大高校"""
                    + major_year
                    + """年不同学位的毕业生人数，请对这份数据做一个评论，具体要求如下：
                    1. 首先数据分为6大块，分别包含"""
                    + rank_year
                    + """年USNews美国大学排名1-50,51-100,101-150,151-200,201-250以及251-300排名区间的有该专业毕业生的大学。
                    2. 其次，每个排名区间下面包含各大学的该年份毕业生人数，其中"ename"为大学英文名，"fpus"为大学链接url，"rank"为"""
                    + rank_year
                    + """年USNews美国大学排名，"count"为该校该专业毕业生总人数，
                        "count0"为该专业该校副学士毕业人数，"count1"为该专业该校本科毕业人数，"count2"为该专业该校硕士毕业人数，"count3"为该专业该校博士毕业人数。
                    3. 请对这份数据做一个综合的分析评论，评论中请挑选一些重点大学评价，比如毕业生人数比较多的大学或者排名比较高的大学，或者有其他特点的大学等等，评论的字数在600字左右。
                    4. 对评论中出现的美国大学一定要加上链接，链接名称为学校英文名"ename"，链接url为"https://www.forwardpathway.us/fpus"，其中"fpus"为数据中该大学"fpus"项。
                    5. 评论输出的格式不要包含标题，只用段落的方式输出内容。
                    6. 最终输出结果用纯英文的方式输出，the final output should be in purely English.""",
                ),
                (
                    "human",
                    "专业中文名：{major_cname}，专业英文名：{major_ename}\n\n具体数据：{data}",
                ),
            ]
        )
        students_chain = students_prompt | llm | StrOutputParser()
        response = students_chain.invoke(
            {
                "major_cname": major_cname,
                "major_ename": major_ename,
                "data": data,
            }
        )
        formatted_response = markdown.markdown(response).replace("</p>\n<p>", "</p><p>")
    except:
        formatted_response = None
    return {"des3_en": formatted_response}

In [11]:
def update_database(state):
    index = state["index"]
    cipcode = state["cipcode"]
    des = state["des"]
    des2 = state["des2"]
    des3 = state["des3"]
    des_en = state["des_en"]
    des2_en = state["des2_en"]
    des3_en = state["des3_en"]
    query = "UPDATE fp_IPEDS.major_translation SET `des`=%s,`des2`=%s,`des3`=%s,`des_en`=%s,`des2_en`=%s,`des3_en`=%s WHERE `CIPCODE`=%s"
    cursor.execute(query, (des, des2, des3, des_en, des2_en, des3_en, cipcode))
    connection.commit()
    print(index, "-", cipcode, "- done!")

In [12]:
workflow = StateGraph(cipcodeState)
workflow.add_node("initial_node", get_cipcode_name)
workflow.add_node("des_node", get_major_des)
workflow.add_node("des_en_node", get_major_des_en)
workflow.add_node("des2_node", major_students_comments)
workflow.add_node("des2_en_node", major_students_comments_en)
workflow.add_node("des3_node", major_colleges_comments)
workflow.add_node("des3_en_node", major_colleges_comments_en)
workflow.add_node("update_node", update_database)

workflow.add_edge(START, "initial_node")
workflow.add_edge("initial_node", "des_node")
workflow.add_edge("initial_node", "des_en_node")
workflow.add_edge("initial_node", "des2_node")
workflow.add_edge("initial_node", "des2_en_node")
workflow.add_edge("initial_node", "des3_node")
workflow.add_edge("initial_node", "des3_en_node")

workflow.add_edge("des_node", "update_node")
workflow.add_edge("des_en_node", "update_node")
workflow.add_edge("des2_node", "update_node")
workflow.add_edge("des2_en_node", "update_node")
workflow.add_edge("des3_node", "update_node")
workflow.add_edge("des3_en_node", "update_node")

workflow.add_edge("update_node", END)
app = workflow.compile()

In [ ]:
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles
display(
    Image(
        app.get_graph().draw_mermaid_png(
            curve_style=CurveStyle.BASIS,
            node_colors=NodeStyles(first="fill:#FDFFB6",last="fill:#FFADAD",default="fill:#CAFFBF,line-height:3"),
            draw_method=MermaidDrawMethod.API,
        )
    )
)
img = app.get_graph().draw_mermaid_png(
    curve_style=CurveStyle.BASIS,
    node_colors=NodeStyles(
        first="fill:#FDFFB6",
        last="fill:#FFADAD",
        default="fill:#CAFFBF,line-height:1",
    ),
    draw_method=MermaidDrawMethod.API,
)
with open("cipcode_update.png", "wb") as png:
    png.write(img)

In [13]:
majors = get_cipcode_list()

In [ ]:
connection = pymysql.connect(
    db=os.environ["db_name"],
    user=os.environ["db_user"],
    passwd=os.environ["db_pass"],
    host=os.environ["db_host"],
    port=3306,
    cursorclass=pymysql.cursors.DictCursor,
)
cursor = connection.cursor()

for i in range(majors.shape[0]):
    app.invoke({"index": i})

cursor.close()
connection.close()